In [42]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB, ComplementNB

1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [4]:
tfidfvect = TfidfVectorizer()

In [36]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [8]:
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [6]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [7]:
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [11]:
rng = np.random.default_rng()
documents_idxs = rng.integers(low=0, high=X_train.shape[0], size=5)
print(documents_idxs)

[  332  3200  4655 10239  7277]


In [24]:
docs_cossims = {}

for idx in documents_idxs:
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    docs_cossims[idx] = {
        'cossims': np.sort(cossim)[::-1][1:6],
        'cossims_idxs': np.argsort(cossim)[::-1][1:6]
    }

print(docs_cossims)

{np.int64(332): {'cossims': array([0.25620852, 0.23991341, 0.23672985, 0.22440601, 0.22165486]), 'cossims_idxs': array([5423, 8224,  913, 2077, 6894])}, np.int64(3200): {'cossims': array([0.29795011, 0.26520154, 0.26033696, 0.25430268, 0.25198717]), 'cossims_idxs': array([10301,  3596,  4271,  9623,  3435])}, np.int64(4655): {'cossims': array([0.40105673, 0.34157183, 0.33164316, 0.30884002, 0.3070807 ]), 'cossims_idxs': array([ 5200, 10924, 10836,  7539,  2095])}, np.int64(10239): {'cossims': array([0.19206856, 0.18971667, 0.18918904, 0.18439022, 0.18296171]), 'cossims_idxs': array([1194, 5489, 8166, 4370, 7722])}, np.int64(7277): {'cossims': array([0.20274257, 0.19575562, 0.15456404, 0.13635304, 0.13313526]), 'cossims_idxs': array([9961, 7998, 3160,  780, 4021])}}


In [30]:
for doc_idx, cossims in docs_cossims.items():
    print("="*50)
    print(f"Clase del documento \n{newsgroups_train.target_names[y_train[doc_idx]]}\n")
    print("Clases de los documentos similares\n")
    for idx, cossim_idx in enumerate(cossims['cossims_idxs']):
        print(f"Clase {newsgroups_train.target_names[y_train[cossim_idx]]} Similaridad: {cossims['cossims'][idx]:.4f}")

Clase del documento 
talk.politics.mideast

Clases de los documentos similares

Clase talk.politics.mideast Similaridad: 0.2562
Clase talk.politics.mideast Similaridad: 0.2399
Clase alt.atheism Similaridad: 0.2367
Clase talk.politics.mideast Similaridad: 0.2244
Clase talk.politics.guns Similaridad: 0.2217
Clase del documento 
talk.politics.mideast

Clases de los documentos similares

Clase talk.politics.mideast Similaridad: 0.2980
Clase talk.politics.misc Similaridad: 0.2652
Clase talk.politics.misc Similaridad: 0.2603
Clase talk.politics.mideast Similaridad: 0.2543
Clase talk.politics.mideast Similaridad: 0.2520
Clase del documento 
alt.atheism

Clases de los documentos similares

Clase alt.atheism Similaridad: 0.4011
Clase alt.atheism Similaridad: 0.3416
Clase alt.atheism Similaridad: 0.3316
Clase alt.atheism Similaridad: 0.3088
Clase alt.atheism Similaridad: 0.3071
Clase del documento 
comp.sys.mac.hardware

Clases de los documentos similares

Clase comp.sys.ibm.pc.hardware Similari

In [35]:
for doc_idx, cossims in docs_cossims.items():
    print("="*100)
    print(f"Documento \n{newsgroups_train.data[doc_idx]}\n")
    print("Documentos similares\n")
    for idx, cossim_idx in enumerate(cossims['cossims_idxs']):
        print("-"*100)
        print(f"Similaridad: {cossims['cossims'][idx]:.4f}\n")
        print(f"{newsgroups_train.data[cossim_idx]}")

Documento 
With regards to my condemnation of Marc's ridiculous attacks on the
American Department of Justice, and further attacks on Jews, to
anyone who took offense to my calling Marc stupid, I
apologize for pointing out the obvious.  It was a waste of the
Net's time.  I hope, though, that most American citizens have
the basic knowlege of the structure of American government to
understand the relationship between the Justice Department
as a part of the Executive Branch, and the Courts, which
are of the Judicial Branch.  
Marc's ignorance of basic civic knowlege underscores his
inability to comprehend and interpret foreign affairs.  


Documentos similares

----------------------------------------------------------------------------------------------------
Similaridad: 0.2562



Anytime.


Suffering from a severe case of myopia? No Muslim left alive - not a 
single one. Leading the first Armenian units who crossed the Ottoman 
border in the company of the Russian invaders was the form

En general, con un vectorizador TF-IDF sin ajustar parámetros, se observa una buena relación de similaridad encontrado con los documentos encontrados. Siendo el mejor el caso de la clase alt.atheism donde las similaridades son más altas que en el resto y los 5 documentos más similares son de esta misma clase. Y la peor performance se encuentra con el documento de la clase rec.sport.hockey. Que aunque encontró similaridad con documentos relacionados a deportes como rec.sport.baseball, la realidad es que al tener tan poco contenido (solo una oración) no encuentra alta similaridad o comete errores, por ejemplo, uno de los documentos de rec.sport.baseball solo dice "was...". Con documentos más extensos se nota que aunque no sean de la misma clase algo de similares tienen, por ejemplo, en el primer documento de la clase talk.politics.mideast, aunque encontró similaridad con documentos de clases diferentes (alt.atheism y talk.politics.guns), al leerlos con algo de detenimiento se observa que la religión y conflictos armados están presentes en todos.

2. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

In [38]:
similarities = cosine_similarity(X_test, X_train)

In [39]:
y_pred = []
for i in range(similarities.shape[0]):
    idx_most_similar = np.argmax(similarities[i])  # índice del documento más parecido en train
    y_pred.append(y_train[idx_most_similar])

In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.37      0.51      0.43       319
           1       0.54      0.48      0.51       389
           2       0.51      0.46      0.48       394
           3       0.52      0.52      0.52       392
           4       0.53      0.50      0.52       385
           5       0.70      0.59      0.64       395
           6       0.63      0.46      0.53       390
           7       0.41      0.58      0.48       396
           8       0.63      0.52      0.57       398
           9       0.65      0.54      0.59       397
          10       0.75      0.72      0.73       399
          11       0.55      0.59      0.57       396
          12       0.53      0.33      0.41       393
          13       0.65      0.49      0.56       396
          14       0.64      0.51      0.57       394
          15       0.45      0.58      0.51       398
          16       0.45      0.47      0.46       364
          17       0.36    

3. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

In [128]:
vectorizer_nb = TfidfVectorizer(
    ngram_range=(1,2), # incluir bigramas
    min_df=3, # filtrar términos muy raros
    max_df=0.9, # filtrar términos muy frecuentes
    stop_words="english", # quitar stopwords en inglés
    strip_accents='ascii', # elimina acentos y caracteres especiales
    sublinear_tf=True, # suaviza los conteos usando log(1+tf)
)
X_train_nb = vectorizer_nb.fit_transform(newsgroups_train.data)
X_test_nb = vectorizer_nb.transform(newsgroups_test.data)
y_train_nb = newsgroups_train.target
y_test_nb = newsgroups_test.target

In [129]:
multinb_clf = MultinomialNB()
multinb_clf.fit(X_train_nb, y_train_nb)

MultinomialNB()

In [130]:
y_pred_multinb = multinb_clf.predict(X_test_nb)

In [131]:
print(classification_report(y_test_nb, y_pred_multinb))

              precision    recall  f1-score   support

           0       0.78      0.17      0.27       319
           1       0.67      0.70      0.68       389
           2       0.68      0.59      0.63       394
           3       0.62      0.73      0.67       392
           4       0.81      0.64      0.72       385
           5       0.79      0.79      0.79       395
           6       0.77      0.80      0.78       390
           7       0.82      0.71      0.76       396
           8       0.87      0.74      0.80       398
           9       0.90      0.79      0.84       397
          10       0.57      0.93      0.71       399
          11       0.66      0.78      0.72       396
          12       0.71      0.54      0.61       393
          13       0.84      0.76      0.79       396
          14       0.75      0.76      0.76       394
          15       0.36      0.92      0.52       398
          16       0.58      0.70      0.63       364
          17       0.78    

In [132]:
complnb_clf = ComplementNB()
complnb_clf.fit(X_train_nb, y_train_nb)

ComplementNB()

In [133]:
y_pred_complnb = complnb_clf.predict(X_test_nb)

In [134]:
print(classification_report(y_test_nb, y_pred_complnb))

              precision    recall  f1-score   support

           0       0.31      0.44      0.36       319
           1       0.72      0.72      0.72       389
           2       0.71      0.63      0.67       394
           3       0.69      0.72      0.70       392
           4       0.79      0.71      0.75       385
           5       0.84      0.81      0.82       395
           6       0.76      0.81      0.79       390
           7       0.81      0.72      0.76       396
           8       0.85      0.77      0.81       398
           9       0.91      0.85      0.88       397
          10       0.88      0.93      0.90       399
          11       0.80      0.80      0.80       396
          12       0.70      0.56      0.62       393
          13       0.81      0.81      0.81       396
          14       0.77      0.80      0.78       394
          15       0.54      0.89      0.67       398
          16       0.59      0.73      0.66       364
          17       0.80    

4. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

In [138]:
tfidfvect = TfidfVectorizer(
    stop_words="english", # quitar stopwords en inglés
    strip_accents='ascii', # elimina acentos y caracteres especiales
    sublinear_tf=True, # suaviza los conteos usando log(1+tf)
)

In [139]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
X_test = tfidfvect.transform(newsgroups_test.data)
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [145]:
X_train_T = X_train.T
print(f'Tamaño del vocabulario: {X_train_T.shape[0]}')

Tamaño del vocabulario: 101320


In [142]:
words_to_compare = ["hockey", "military", "bible", "pitch", "computer"]
word_idxs = [tfidfvect.vocabulary_[w] for w in words_to_compare]
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}

In [149]:
cossin_by_word = {}

for idx, word in zip(word_idxs, words_to_compare):
    # vector de la palabra
    word_v = X_train_T[idx]
    # similitud coseno con todas las palabras
    cossim = cosine_similarity(word_v, X_train_T)[0]

    cossin_by_word[word] = {
        "cossims": np.sort(cossim)[::-1][1:6],
        "cossims_idxs": np.argsort(cossim)[::-1][1:6]
    }

In [150]:
for word in words_to_compare:
    similar_words = [idx2word[i] for i in cossin_by_word[word]["cossims_idxs"]]
    print(f"Palabra: {word}")
    print(f"-- Similitud coseno con las 5 más similares: {cossin_by_word[word]['cossims']}")
    print(f"-- Palabras más similares: {similar_words}\n")

Palabra: hockey
-- Similitud coseno con las 5 más similares: [0.27519008 0.22325525 0.20587735 0.20518409 0.20222439]
-- Palabras más similares: ['nhl', 'ncaa', 'players', 'league', 'game']

Palabra: military
-- Similitud coseno con las 5 más similares: [0.21947214 0.21947214 0.21947214 0.19721739 0.19593406]
-- Palabras más similares: ['civie', '62mm', 'fmjs', 'conscription', 'fractl']

Palabra: bible
-- Similitud coseno con las 5 más similares: [0.27439685 0.26668925 0.22622701 0.2113792  0.20667636]
-- Palabras más similares: ['christians', 'god', 'jesus', 'contadictions', 'biblical']

Palabra: pitch
-- Similitud coseno con las 5 más similares: [0.33029491 0.26181059 0.23133711 0.22212085 0.21994078]
-- Palabras más similares: ['curveball', 'mound', 'loosen', 'hibbard', 'crossdominance']

Palabra: computer
-- Similitud coseno con las 5 más similares: [0.15114602 0.12368539 0.12132836 0.12059295 0.12059295]
-- Palabras más similares: ['shopper', 'decwriter', 'shops', 'deluged', 'hark

Los resultados obtenidos muestran que el método logra capturar relaciones semánticas coherentes:
- hockey se asocia con “nhl” y “league”.
- bible con “god” y “jesus”.
- pitch con términos de béisbol.
- military con términos bastante técnicos y específicos (“62mm”, “fmjs” → Full Metal Jacket bullets). Esto sugiere que en el corpus el tema militar está fuertemente relacionado a armamento.

Aunque también, hay términos menos intuitivos como:
- computer con “shopper” y “deluged”, probablemente porque la palabra “computer” está muy distribuida en el corpus y aparece en muchos contextos.

En conclusión, el método funciona bien observando coherencia semántica entre términos cercanos aunque las similitudes son bajas en valor absoluto (≈0.15–0.33), pero eso es común con TF-IDF ya que los vectores son muy esparsos.